# 時系列データのベイズ分析

ベクトルや行列を太文字表記していないので要注意

In [ ]:
# 必要なライブラリをインポート
import numpy as np
import scipy.stats as st
import scipy.linalg as la
import pymc3 as pm
import matplotlib.pyplot as plt
import japanize_matplotlib
%matplotlib inline

import warnings
warnings.simplefilter('ignore')

## 時系列データと状態空間表現
時系列データの大きな特徴<br>
-> 現在の観測値の分布が過去の観測値に依存する（独立でない）

時点$t$の観測値を$y_t(t=1, \dotsc ,n)$とし，$y_{1:t}=(y_1, \dotsc ,y_t)$とまとめて表記する．<br>
このとき，$y_t$の分布が過去の観測値$y_{1: t-1}$に依存すると仮定すると，$y_{1:n}$の同時分布は下記の通り．
$$
    p(y_{1:n}|\theta) = p(y_1|\theta)p(y_2|y_1, \theta) \times \cdots \times p(y_n|y_{1:n-1}, \theta), 
$$

時系列データ$y_{1:n}$の挙動を説明する最も基礎的な状態空間モデルは，下記の2つの式から構成される．
$$
    y_t = Z_t \alpha_t + \epsilon_t, \tag{1}
$$
$$
    \alpha_{t+1} = T_t \alpha_t + \eta_t, \tag{2}
$$

(1)式は**観測方程式**，(2)式は**状態方程式**と呼ばれる．

|変数|概要|次元|仮定|
|-|-|-|-|
|$y_t$|観測される時系列データ|$m \times 1$ベクトル|ー|
|$\alpha_t$|状態変数．観測されない確率変数|$r \times 1$ベクトル|$\alpha_1 \sim N_r(T_0\alpha_0, H_0H_0^T)$|
|$\epsilon_t$|観測方程式の誤差項|$m \times 1$ベクトル|$e_t=G_tu_t$|
|$\eta_t$|状態方程式の誤差項|$r \times 1$ベクトル|$\eta_t=H_tu_t$|
|$u_t$|観測されない確率変数|$l \times 1$ベクトル|$u_t \sim N_l(0,I)$|
|$Z_t$|ー|$m \times r$行列|ー|
|$G_t$|ー|$m \times l$行列|ー|
|$T_t$|ー|$r \times r$行列|ー|
|$H_t$|ー|$r \times l$行列|ー|

以上をまとめると状態空間モデルは下記の通りに書き直すことができる．
$$
    y_t = Z_t \alpha_t + G_t u_t, \tag{3} \\
    \alpha_{t+1} = T_t \alpha_t + H_t u_t, 
$$

さらに，$\{Z_t\}_{t=1}^n, \{G_t\}_{t=1}^n, \{T_t\}_{t=0}^n, \{H_t\}_{t=0}^n, \alpha_0$が全て既知であると仮定すると，(3)式における$\alpha_t$を所与としたときの$y_t$と$\alpha_{t+1}$の同時分布は下記の通りになる．
$$
    \begin{bmatrix}
        y_t \\ \alpha_{t+1}
    \end{bmatrix} \bigg| \alpha_t \sim
    N_{m+r} \bigg( 
        \begin{bmatrix}
            Z_t \alpha_t \\ T_t \alpha_t
        \end{bmatrix}, \ 
        \begin{bmatrix}
            G_tG_t^T && G_tH_t^T \\
            H_tG_t^T && H_tH_t^T
        \end{bmatrix}
    \bigg), \tag{4}
$$

## 状態空間モデルに関する推論
